<a href="https://colab.research.google.com/github/aknroshn/Image-Classification-Using-Transfer-Learning/blob/main/DL_TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Import ImageDataGenerator from tensorflow.keras.preprocessing.image
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
#from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD


In [ ]:
# Step 1: Create the base pre-trained InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# Step 2: Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling to reduce dimensions
x = Dense(1024, activation='relu')(x)  # Fully connected layer with ReLU activation
num_classes = 2  # Adjust based on your dataset
predictions = Dense(num_classes, activation='softmax')(x)  # Output layer for multi-class classification

In [ ]:
# Step 3: Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Step 4: Freeze all the layers in the base InceptionV3 model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Step 5: Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [ ]:
# Step 6: Load and preprocess the dataset using ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,        # Normalize pixel values to [0,1]
    shear_range=0.2,          # Apply shear transformations
    zoom_range=0.2,           # Apply random zoom
    horizontal_flip=True      # Randomly flip images horizontally
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1.0/255.0)  # For validation/test set, just rescale


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Step 7: Load the training and validation sets from directories
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Train',
    target_size=(299, 299),   # InceptionV3 expects images of size 299x299
    batch_size=32,
    class_mode='categorical'  # Categorical mode for multi-class classification
)
validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Test',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

Found 10 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


In [ ]:
# Step 8: Train the model on the new dataset for a few epochs
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 19s 19s/step - loss: 0.6575 - val_loss: 0.9982
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 0.6422 - val_loss: 14.4643
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: 12.0781 - val_loss: 0.2849
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: 0.0132 - val_loss: 0.3385
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - loss: 0.0082 - val_loss: 0.2948
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - loss: 0.0172 - val_loss: 0.2586
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - loss: 0.0252 - val_loss: 0.5437
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 0.0236 - val_loss: 0.2489
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 0.0843 - val_loss: 2.0645
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: 0.3230 - val_loss: 3.3737


In [ ]:
# Step 9: Fine-tuning - Unfreeze top layers for further training
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_layer
1 conv2d
2 batch_normalization
3 activation
4 conv2d_1
5 batch_normalization_1
6 activation_1
7 conv2d_2
8 batch_normalization_2
9 activation_2
10 max_pooling2d
11 conv2d_3
12 batch_normalization_3
13 activation_3
14 conv2d_4
15 batch_normalization_4
16 activation_4
17 max_pooling2d_1
18 conv2d_8
19 batch_normalization_8
20 activation_8
21 conv2d_6
22 conv2d_9
23 batch_normalization_6
24 batch_normalization_9
25 activation_6
26 activation_9
27 average_pooling2d
28 conv2d_5
29 conv2d_7
30 conv2d_10
31 conv2d_11
32 batch_normalization_5
33 batch_normalization_7
34 batch_normalization_10
35 batch_normalization_11
36 activation_5
37 activation_7
38 activation_10
39 activation_11
40 mixed0
41 conv2d_15
42 batch_normalization_15
43 activation_15
44 conv2d_13
45 conv2d_16
46 batch_normalization_13
47 batch_normalization_16
48 activation_13
49 activation_16
50 average_pooling2d_1
51 conv2d_12
52 conv2d_14
53 conv2d_17
54 conv2d_18
55 batch_normalization_12
56 batch_normalization_

In [ ]:
# Unfreeze the top layers after the first 249 layers for fine-tuning
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

In [ ]:
# Step 10: Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy') # Use learning_rate instead of lr
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10  # Continue training for additional epochs with fine-tuning
)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step - loss: 3.4161 - val_loss: 3.3362
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - loss: 3.1543 - val_loss: 3.2759
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 3.1944 - val_loss: 3.1955
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: 3.1979 - val_loss: 3.0977
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: 3.3204 - val_loss: 2.9836
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - loss: 2.9175 - val_loss: 2.8622
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - loss: 2.7429 - val_loss: 2.7311
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: 2.2593 - val_loss: 2.5983
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 2.0800 - val_loss: 2.4594
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - loss: 1.8902 - val_loss: 2.3183


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
def predict_image(image_path, model):


  img = image.load_img(image_path, target_size=(299, 299))
  img_array = image.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array /= 255.0  # Normalize the image

  prediction = model.predict(img_array)
  predicted_class = np.argmax(prediction)
  class_probability = prediction[0][predicted_class]


  return predicted_class, class_probability

In [ ]:
uploaded_image_path = '/content/drive/MyDrive/Test/truck/09797.jpeg'
predicted_label, probability = predict_image(uploaded_image_path, model)

print(f"Predicted label: {predicted_label}, Probability: {probability}")

class_names = list(train_generator.class_indices.keys())
if predicted_label < len(class_names):
  predicted_class_name = class_names[predicted_label]
  print(f"Predicted class name: {predicted_class_name}")
else:
  print("Predicted label is out of range of class names.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted label: 1, Probability: 0.9999949932098389
Predicted class name: truck
